# 手書き文字認識を行うNeural Networkのレシピ

## このスクリプトについて
ニューラルネットワークを使った画像認識のレシピです。今回はチュートリアルなどでよく使われている手書き文字データセットMNISTを学習してモデルを作成し、テスト画像を入力してその認識精度を確認します。

## 実験条件の設定

実験で用いるデータセットのサイズを指定します。MNIST(FashionMNIST)は60000枚の画像を含むデータセットですが、ここで指定した割合のデータを学習・評価に用います。

In [ ]:
# 20% -> 0.2
util_rate = 0.2

## 環境のセットアップ


### GPUの種類を確認する
スピード　：　Tesla K80 < Tesla T4 < Tesla P100

In [ ]:
!nvidia-smi

### 必要なライブラリのインポート

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

### GPUが有効になっているか確認
CUDAはGPUを供給するメーカーのnVIDIA社が提供するドライバである。CPUと表示された場合はランタイムのタイプをGPUに変更してライブラリをもう一度インポートする。

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

## ニューラルネットワークのパラメータと学習の設定

ダウンロードエラーになる場合は8行目の先頭に#を追加して、9行目の先頭の#を削除してください。  
(コメントアウト)

In [ ]:
# 前処理
transform = transforms.Compose([
    # 画像をTensorに変換してくれる
    # チャネルラストをチャネルファーストに
    # 0〜255の整数値を0.0〜1.0の浮動小数点に変換してくれる
    transforms.ToTensor()                              
])
dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
#dataset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)

#実験条件で設定した割合のデータをdatasetから取り出す
indices = np.random.permutation(len(dataset))[:int(len(dataset)*util_rate)]
subset = torch.utils.data.Subset(dataset, indices)

#取り出したdatasetのうち80%を学習に、20%をテストに使う。
n_imgs = len(subset)
n_train = int(len(subset)*0.8)
n_val = n_imgs - n_train

train_dataset, val_dataset = torch.utils.data.random_split(subset, [n_train, n_val])

In [ ]:
num_batches = 100
if len(train_dataset)%100 != 0 or len(val_dataset)%100 != 0:
  num_batches = 10
print('Batch Size : ', num_batches)
train_dataloader = DataLoader(train_dataset, batch_size=num_batches, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=num_batches, shuffle=False)
train_iter = iter(train_dataloader)
# 100個だけミニバッチからデータをロードする
imgs, labels = train_iter.next()

## trainingに用いる画像の確認

In [ ]:
print(labels)

In [ ]:
img = imgs[0]
# 画像データを表示するために、チャネルファーストのデータをチャネルラストに変換する
img_permute = img.permute(1, 2, 0)
# tensorから2次元のarrayに変換する
sns.heatmap(img_permute.numpy()[:, :, 0])

## ニューラルネットワークの定義

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 100)
        self.relu1 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(100, 100)
        self.relu2 = nn.ReLU(inplace=True)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)

        return x

In [ ]:
model = MLP()

print(model.to(device))

## ニューラルネットワークのトレーニング

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10
train_losses = []
train_accs = []
val_losses = []
val_accs = []

start_time = time.process_time()

for epoch in range(num_epochs):
    #train
    running_train_loss = 0.0
    running_train_acc = 0.0
    for imgs, labels in train_dataloader:
        imgs = imgs.view(num_batches, -1)
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        output = model(imgs)
        loss = criterion(output, labels)
        running_train_loss += loss.item()
        # dim=1 => 0-9の分類方向のMax値を返す
        pred = torch.argmax(output, dim=1)
        running_train_acc += torch.mean(pred.eq(labels).float())
        loss.backward()
        optimizer.step()
    # 画像数で割る
    running_train_loss /= len(train_dataloader)
    running_train_acc /= len(train_dataloader)
    train_losses.append(running_train_loss)
    train_accs.append(running_train_acc)

    #test
    running_val_loss = 0.0
    running_val_acc = 0.0
    with torch.no_grad():
      for imgs, labels in val_dataloader:
        imgs = imgs.view(num_batches, -1)
        imgs = imgs.to(device)
        labels = labels.to(device)
        output = model(imgs)
        loss = criterion(output, labels)
        running_val_loss += loss.item()
        # dim=1 => 0-9の分類方向のMax値を返す
        pred = torch.argmax(output, dim=1)
        running_val_acc += torch.mean(pred.eq(labels).float())
        
    # 画像数で割る
    running_val_loss /= len(val_dataloader)
    running_val_acc /= len(val_dataloader)
    val_losses.append(running_val_loss)
    val_accs.append(running_val_acc)

    print("epoch: {}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(epoch, running_train_loss, running_train_acc, running_val_loss, running_val_acc))

end_time = time.process_time()


## 結果の確認



Loss：正しい答えとニューラルネットワーク出力の誤差  
Accuracy：ニューラルネットワーク推論結果の正解率

### 学習データ

Loss:

In [ ]:
plt.plot(train_losses)

Accuracy:

In [ ]:
plt.plot(train_accs)

### テストデータ

Loss:

In [ ]:
plt.plot(val_losses)

Accuracy:

In [ ]:
plt.plot(val_accs)

## Summary

In [ ]:
!nvidia-smi
print('Accuracy:{:.04f}'.format(running_val_acc))
print('Time cost:{:.04f}'.format(end_time-start_time), 's')

## Appendix
ニューラルネットワークの推論が間違えていた画像を確認する。  
最大10枚まで表示

In [ ]:
demo_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)
max_images = 10
with torch.no_grad():
  for imgs, labels in demo_dataloader:
    imgs_reshape = imgs.view(1, -1)
    imgs_reshape = imgs_reshape.to(device)
    labels = labels.to(device)
    output = model(imgs_reshape)
    # dim=1 => 0-9の分類方向のMax値を返す
    pred = torch.argmax(output, dim=1)
    if not labels==pred:
      max_images -= 1
      print('correct: ', int(labels))
      print('predicted: ', int(pred))
      for i in range(28):
        for j in range(28):
          if imgs_reshape[0][i*28+j]==0:
            print('□', end='')
          else:
            print('■', end='')
        print('')
      print('')
    if max_images == 0:
      break
